In [1]:
import sys
sys.path.append('M:/comlib/')
from utilities import *
from datetimerange import DateTimeRange
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

from functools import reduce 
import pandas as pd
import re
from glob import glob 
import collections
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

# import tf.keras as keras

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
# config = tf.ConfigProto(
#         device_count = {'GPU': 0}
#     )
set_session(tf.Session(config=config))

from pathlib import Path
homepath = str(Path.home())
save_dir  = f'{homepath}/Downloads/'


K:\anaconda3\envs\stlf_r114\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
import numpy as np
from datetime import datetime
import pandas as pd
All = pd.read_csv('M:/Projects/STLF/ProfileExact15/Data/PreparedData.csv',sep = ',',parse_dates =['Timestamp'], index_col=['Timestamp'])

xc=All[datetime(2018,12,1,0,0,0):]


In [16]:
xc=xc.reset_index()
xc=xc.loc[:(xc.shape[0]//96)*96-1,:]

ats=pd.Series(pd.to_datetime(xc["Timestamp"][:(All.shape[0]//96)*96]).apply(lambda x: x.date()).unique(), name='date')
ats.sort_values(ascending=True, inplace=True)


In [3]:
# checking against the old model
testing_dates=[]
testing_values=[]
for i in glob('m:Projects/STLF/ProfileExact15/Reports/*_new.csv'):
    testing_dates.append(datetime.strptime(i.split('\\')[-1].split('_')[0], '%Y%m%d').date())
    testing_values.append(pd.read_csv(i).get_values().mean(0)/8000)
#     print(i, (pd.read_csv(i).iloc[:,1:].get_values().mean(0)/8000).shape)
#     print(len(testing_values))
#     print(pd.read_csv(i).shape)
#     break
testing_dates=pd.Series(testing_dates, name='date')
testing_values=np.concatenate(testing_values).reshape(-1,96)
backtest1=pd.DataFrame({'date':testing_dates, 'values':testing_values.tolist()}).set_index('date')

In [7]:
backtemp=pd.read_csv('m:Projects/STLF/ProfileExact15/Reports/stlf_backtest.csv')

# backtemp
backtemp_date=pd.Series(pd.to_datetime(backtemp["timestamp"][:(backtemp.shape[0]//96)*96]).apply(lambda x: x.date()).unique(), name='date')
backtemp_values=(backtemp['predict'].get_values().reshape(-1,96)/8000)
backtest2=pd.DataFrame({'date':backtemp_date, 'values':backtemp_values.tolist()}).set_index('date')
overlap=backtest1.join(backtest2, how='inner',lsuffix=('_a')).reset_index()
# np.isnan(backtest['values_a'])

In [19]:
backtest_all=backtest2[ats.min(): ats.max()].append(backtest1[overlap.date.max()+timedelta(days=1):ats.max()])
# backtest1.date.min(), 
oldbacktesting=np.concatenate(backtest_all['values'].get_values().tolist(), axis=0).reshape(-1,96)

In [20]:
oldbacktesting.shape

(324, 96)

In [22]:
from datetime import time
D=xc[(xc['Timestamp']>=datetime.combine(ats.min(), time(0,0,0))) & (xc['Timestamp']<=datetime.combine(ats.max(), time(23,59,59)))]['EMS.MUD..OPERATIONAL.DEMAND.MW']\
.get_values().reshape(-1, 96)/8000

In [24]:
oldbacktesting.shape, D.shape

((324, 96), (324, 96))

In [69]:
import tensorflow as tf
import keras.backend as K
def funcg(x):
    def genmask2(x):
        tfact=x[0]
        tff2=x[1]
        w=8
        a=tf.expand_dims(tfact, axis=0)
        b=tf.expand_dims(tff2, axis=1)
        xxx=tf.abs(a-b)
        mask=tf.matrix_band_part(tf.ones(xxx.shape), w,w)
        mask2=(tf.ones(xxx.shape)-mask)*9
        masked=mask*xxx+mask2
        masked_copy=mask*xxx+mask2

        masked_copy -= K.min(masked_copy, axis=1)[..., np.newaxis]
        masked_copy -= K.min(masked_copy, axis=0)[np.newaxis, ...]

        maskc=tf.ones(xxx.shape[0])
        mask_eye=tf.matrix_band_part(tf.ones(xxx.shape), 0,0)*9
        y=[]
        y2=[]
        mask_temp=tf.zeros(xxx.shape[0])
        y_=tf.constant([], dtype=tf.int64)
        for i in range(xxx.shape[1]):
            y_=tf.argmin(masked_copy[i]+mask_temp, axis=0)
            y__=tf.constant(i, dtype=tf.int64)
            mask_temp=mask_eye[y_]+mask_temp
            y.append(y_)
            y2.append(y__)
        ind=tf.stack([tf.stack(y2, axis=0), tf.stack(y, axis=0)], axis=1)
        err=tf.gather_nd(masked, ind)
        err2=tf.abs(tfact-tff2)
        comberr=tf.minimum(err, err2)
        ans=K.sqrt(K.sqrt(K.sum(K.square(K.square(comberr)))))
        return ans
    res = K.map_fn(genmask2, x, dtype=tf.float32)
    return res

tfact=tf.placeholder(shape=(None,96), dtype=tf.float32)
tff2=tf.placeholder(shape=(None, 96,), dtype=tf.float32)
# fff  = tf.stack((tfact, tff2), axis=1)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    fr=sess.run(funcg((tfact, tff2)),  feed_dict={tfact: oldbacktesting, tff2:  D} )
    
fr

array([0.02565916, 0.0277051 , 0.0120152 , 0.02047683, 0.02964486,
       0.02966449, 0.02704143, 0.02499739, 0.02104303, 0.02779756,
       0.01744389, 0.01821008, 0.02551523, 0.02248381, 0.01803783,
       0.01317921, 0.01509254, 0.02155441, 0.01330766, 0.01308796,
       0.01741322, 0.01952618, 0.04058489, 0.19420642, 0.06311107,
       0.08188409, 0.02094609, 0.02066218, 0.08747902, 0.06752083,
       0.12606159, 0.07171959, 0.02275714, 0.02188334, 0.04784394,
       0.02529994, 0.01752584, 0.02619291, 0.01349445, 0.01442293,
       0.01376909, 0.01751853, 0.02273732, 0.00917892, 0.01898414,
       0.02595236, 0.01722783, 0.03007411, 0.02051779, 0.01151912,
       0.02218892, 0.0297932 , 0.01574932, 0.01620661, 0.01532477,
       0.017651  , 0.02108934, 0.01421622, 0.01762991, 0.01299976,
       0.01765939, 0.0147919 , 0.0389664 , 0.0418776 , 0.05738051,
       0.09257298, 0.16047093, 0.11611795, 0.08714242, 0.09623305,
       0.05610775, 0.04755826, 0.01768102, 0.03055007, 0.02768

In [70]:
fr.shape

(324,)